In [20]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets
%matplotlib widget
from plotly.offline import init_notebook_mode, plot

In [34]:
# loading data right from the source:

death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [54]:
confirmed_df=confirmed_df.dropna(subset=['lat', 'long'])

In [55]:
confirmed_df.isnull().sum()

state      189
country      0
lat          0
long         0
1/22/20      0
          ... 
3/14/21      0
3/15/21      0
3/16/21      0
3/17/21      0
3/18/21      0
Length: 426, dtype: int64

In [56]:
# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})

In [57]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [58]:
display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )


In [59]:
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)
print("Change Value of N")
interact(show_latest_cases, n='10')

Change Value of N


interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

<function __main__.show_latest_cases(n)>

In [60]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [61]:
def bubble_chart(n):
    if n>=1:
        fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
                   hover_name="country", size_max=60)
        fig.update_layout(
        title=str(n) +" Worst hit countries",
        xaxis_title="Countries",
        yaxis_title="Confirmed Cases",
        width = 850
        )
        fig.show();
    else:
        print("invalid Input")

interact(bubble_chart, n=10)


interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.bubble_chart(n)>

In [62]:

sorted_country_on_death_df = country_df.sort_values('deaths', ascending= False)
def bubble_chart_on_death(n):
    if n>=1:
        fig = px.scatter(sorted_country_on_death_df.head(n), x="country", y="deaths", size="deaths", color="country",
                   hover_name="country", size_max=60)
        fig.update_layout(
        title=str(n) +" Worst hit on death countries",
        xaxis_title="Countries",
        yaxis_title="Deaths Cases",
        width = 850
        )
        fig.show();
    else:
        print("invalid Input")

interact(bubble_chart_on_death, n=10)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.bubble_chart_on_death(n)>

In [63]:
sorted_country_on_recovered_df = country_df.sort_values('recovered', ascending= False)
def bubble_chart_on_recovered(n):
    if n>=1:
        fig = px.scatter(sorted_country_on_recovered_df.head(n), x="country", y="recovered", size="recovered", color="country",
                   hover_name="country", size_max=60)
        fig.update_layout(
        title=str(n) +" Highest recovry seen countries",
        xaxis_title="Countries",
        yaxis_title="recovered Cases",
        width = 850
        )
        fig.show();
    else:
        print("invalid Input")

interact(bubble_chart_on_recovered, n=10)


interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.bubble_chart_on_recovered(n)>

In [64]:
def plot_cases_of_a_country(country):
    country = country.capitalize()
    labels = ['confirmed', 'deaths','recovered']
    colors = ['blue', 'red','green']
    mode_size = [6, 8,1]
    line_size = [4, 5, 1]

    df_list = [confirmed_df, death_df,recovered_df]
    
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,20:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 850,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

In [65]:
interact(plot_cases_of_a_country, country='India')

interactive(children=(Text(value='India', description='country'), Output()), _dom_classes=('widget-interact',)…

<function __main__.plot_cases_of_a_country(country)>

In [66]:
def plot_daily_cases_of_a_country(country):
    country = country.capitalize()
    labels = ['confirmed']
    colors = ['blue']
    mode_size = [6]
    line_size = [4]

    df_list = [confirmed_df]
    
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,20:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
        y_daily = []
        y_daily.append(y_data[0])
        for j in range(1,len(y_data)):
            y_daily.append(y_data[j] - y_data[j-1])
        fig.add_trace(go.Bar(
            x=x_data,
            y=y_daily,
            name='confirmed cases',
            marker_color='red'
        ))
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 850,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();
interact(plot_daily_cases_of_a_country, country='world')


interactive(children=(Text(value='world', description='country'), Output()), _dom_classes=('widget-interact',)…

<function __main__.plot_daily_cases_of_a_country(country)>

In [67]:
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map

In [68]:
!pip install voila

  Using cached https://files.pythonhosted.org/packages/45/a4/4c79fec6bd6585cba34892824e057273b9256936eb5f85421bc4bd4da616/voila-0.2.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/13/2f/acbe7006548f3914456ee47f97a2033b1b2f3daf921b12ac94105d87c163/nbconvert-6.0.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/22/a6/f3a01a5c1a0e72d1d064f33d4cd9c3a782010f48f48f47f256d0b438994a/nbclient-0.5.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ec/b7/f8964ca33e61c6fa8f4e494e4940fb6de021f5caa52c2dce32316fac8611/jupyter_server-1.4.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a8/6f/c34288766797193b512c6508f5994b830fb06134fdc4ca8214daba0aa443/jupyterlab_pygments-0.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/71/52/39d20e03abd0ac9159c162ec24b93fbcaa111e8400308f2465432495ca2b/async_generator-1.10-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packa

ERROR: spyder 3.3.6 requires pyqt5<5.13; python_version >= "3", which is not installed.
ERROR: spyder 3.3.6 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
ERROR: nbclient 0.5.3 has requirement nbformat>=5.0, but you'll have nbformat 4.4.0 which is incompatible.
ERROR: jupyter-server 1.4.1 has requirement tornado>=6.1.0, but you'll have tornado 5.1.1 which is incompatible.
ERROR: Cannot uninstall 'terminado'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
